<a href="https://colab.research.google.com/github/kuds/rl-connect-four/blob/main/%5BConnect%20Four%5D%20Self%20Play.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Playing Connect Four using Self Play

In [1]:
!pip install https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-3.0.0.dev0-cp311-cp311-manylinux2014_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 25.6 MB/s eta 0:00:00


In [2]:
!pip install gputil open_spiel gymnasium

In [3]:
import functools
import numpy as np
import multiprocessing as mp
import ray
from ray import tune
from ray.air.constants import TRAINING_ITERATION
from ray.rllib.core.rl_module.default_model_config import DefaultModelConfig
from ray.rllib.core.rl_module.multi_rl_module import MultiRLModuleSpec
from ray.rllib.core.rl_module.rl_module import RLModuleSpec
from ray.rllib.env.utils import try_import_pyspiel, try_import_open_spiel
from ray.rllib.env.wrappers.open_spiel import OpenSpielEnv
from ray.rllib.examples.rl_modules.classes.random_rlm import RandomRLModule
from ray.rllib.examples.multi_agent.utils import (
    ask_user_for_action,
    SelfPlayCallback,
    SelfPlayCallbackOldAPIStack,
)
from ray.rllib.examples._old_api_stack.policy.random_policy import RandomPolicy
from ray.rllib.policy.policy import PolicySpec
from ray.rllib.utils.metrics import NUM_ENV_STEPS_SAMPLED_LIFETIME
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import get_trainable_cls, register_env
import platform

import torch
from importlib.metadata import version

2025-01-16 02:18:03,459	WARNING compression.py:16 -- lz4 not available, disabling sample compression. This will significantly impact RLlib performance. To install lz4, run `pip install lz4`.


In [4]:
print(f"Python Version: {platform.python_version()}")
print(f"Torch Version: {version('torch')}")
print(f"Is Cuda Available: {torch.cuda.is_available()}")
print(f"Cuda Version: {torch.version.cuda}")
print(f"Numpy Version: {version('numpy')}")
print(f"Ray Version: {version('ray')}")
print(f"Gymnasium Version: {version('Gymnasium')}")
print(f"Open Spiel Version: {version('open_spiel')}")

Python Version: 3.11.11
Torch Version: 2.5.1+cu121
Is Cuda Available: True
Cuda Version: 12.1
Numpy Version: 1.26.4
Ray Version: 3.0.0.dev0
Gymnasium Version: 1.0.0
Open Spiel Version: 1.5


In [5]:
print(f"Number or CPUs Available: {mp.cpu_count()}")

Number or CPUs Available: 12


In [6]:
open_spiel = try_import_open_spiel(error=True)
pyspiel = try_import_pyspiel(error=True)

# Import after try_import_open_spiel, so we can error out with hints.
from open_spiel.python.rl_environment import Environment  # noqa: E402

In [7]:
class Args:
    def __init__(self):
        self.env = "connect_four"
        self.checkpoint_freq = 1
        self.checkpoint_at_end = True
        self.win_rate_threshold = 0.95
        self.min_league_size = 3
        self.num_episodes_human_play = 1
        self.from_checkpoint = None
        # Add other necessary attributes from parser arguments
        self.algo = 'PPO' # Assuming PPO is the default algorithm
        self.num_env_runners = 2
        self.enable_new_api_stack = True
        self.stop_timesteps = 2000000
        self.stop_iters = 100
        self.as_release_test = False
        self.num_cpus = 10
        self.local_mode = False
        self.framework = 'torch'
        self.num_gpus = 0
        self.num_gpus_per_learner = 1
        self.num_learners = 1
        self.evaluation_interval = 0
        self.log_level = None
        self.output = None
        self.no_tune = False
        self.num_agents = 0
        self.verbose = 2
        self.num_samples = 1
        self.max_concurrent_trials = None
        self.as_test = False
        self.num_envs_per_env_runner = 1 if self.enable_new_api_stack else 5

args = Args()

In [8]:
register_env("open_spiel_env",
             lambda _: OpenSpielEnv(pyspiel.load_game(args.env)))

def agent_to_module_mapping_fn(agent_id, episode, **kwargs):
        # agent_id = [0|1] -> module depends on episode ID
        # This way, we make sure that both modules sometimes play agent0
        # (start player) and sometimes agent1 (player to move 2nd).
        return "main" if hash(episode.id_) % 2 == agent_id else "random"

def policy_mapping_fn(agent_id, episode, worker, **kwargs):
    return "main" if episode.episode_id % 2 == agent_id else "random"

config = (
    get_trainable_cls(args.algo)
    .get_default_config()
    .environment("open_spiel_env")
    # Set up the main piece in this experiment: The league-bases self-play
    # callback, which controls adding new policies/Modules to the league and
    # properly matching the different policies in the league with each other.
    .callbacks(
        functools.partial(
            (
                SelfPlayCallback
                if args.enable_new_api_stack
                else SelfPlayCallbackOldAPIStack
            ),
            win_rate_threshold=args.win_rate_threshold,
        )
    )
    .env_runners(
        num_env_runners=(args.num_env_runners or 2),
        num_envs_per_env_runner=1 if args.enable_new_api_stack else 5,
    )
    .multi_agent(
        # Initial policy map: Random and default algo one. This will be expanded
        # to more policy snapshots taken from "main" against which "main"
        # will then play (instead of "random"). This is done in the
        # custom callback defined above (`SelfPlayCallback`).
        policies=(
            {
                # Our main policy, we'd like to optimize.
                "main": PolicySpec(),
                # An initial random opponent to play against.
                "random": PolicySpec(policy_class=RandomPolicy),
            }
            if not args.enable_new_api_stack
            else {"main", "random"}
        ),
        # Assign agent 0 and 1 randomly to the "main" policy or
        # to the opponent ("random" at first). Make sure (via episode_id)
        # that "main" always plays against "random" (and not against
        # another "main").
        policy_mapping_fn=(
            agent_to_module_mapping_fn
            if args.enable_new_api_stack
            else policy_mapping_fn
        ),
        # Always just train the "main" policy.
        policies_to_train=["main"],
    )
    .rl_module(
        model_config=DefaultModelConfig(fcnet_hiddens=[512, 512]),
        rl_module_spec=MultiRLModuleSpec(
            rl_module_specs={
                "main": RLModuleSpec(),
                "random": RLModuleSpec(module_class=RandomRLModule),
            }
        ),
    )
)

# Only for PPO, change the `num_epochs` setting.
if args.algo == "PPO":
    config.training(num_epochs=20)

stop = {
    NUM_ENV_STEPS_SAMPLED_LIFETIME: args.stop_timesteps,
    TRAINING_ITERATION: args.stop_iters,
    "league_size": args.min_league_size,
}

# Train the "main" policy to play really well using self-play.
results = None
if not args.from_checkpoint:
    results = run_rllib_example_script_experiment(
        config, args, stop=stop
    )

Usage stats collection is enabled by default for nightly wheels. To disable this, run the following command: `ray disable-usage-stats` before starting Ray. See https://docs.ray.io/en/master/cluster/usage-stats.html for more details.


/usr/local/lib/python3.11/dist-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/usr/local/lib/python3.11/dist-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(
/usr/local/lib/python3.11/dist-packages/gymnasium/utils/passive_env_checker.py:134: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/usr/local/lib/python3.11/dist-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
2025-01-16 02:18:06,043	INFO worker.py:1841 -- Started a local Ray instance.
2025-01-16 02:18:06,983	INFO tensorboardx.py:193 -- pip install "ray[tune]" to see T

== Status ==
Current time: 2025-01-16 02:18:07 (running for 00:00:00.56)
Using FIFO scheduling algorithm.
Logical resource usage: 0/10 CPUs, 0/1 GPUs (0.0/1.0 accelerator_type:L4)
Result logdir: /tmp/ray/session_2025-01-16_02-18-04_666737_42841/artifacts/2025-01-16_02-18-06/PPO_2025-01-16_02-18-06/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2025-01-16 02:18:12 (running for 00:00:05.59)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/10 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:L4)
Result logdir: /tmp/ray/session_2025-01-16_02-18-04_666737_42841/artifacts/2025-01-16_02-18-06/PPO_2025-01-16_02-18-06/driver_artifacts
Number of trials: 1/1 (1 PENDING)




(pid=43809) lz4 not available, disabling sample compression. This will significantly impact RLlib performance. To install lz4, run `pip install lz4`.
(PPO pid=43809) 2025-01-16 02:18:15,507	WARNING algorithm_config.py:4702 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html


== Status ==
Current time: 2025-01-16 02:18:17 (running for 00:00:10.60)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/10 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:L4)
Result logdir: /tmp/ray/session_2025-01-16_02-18-04_666737_42841/artifacts/2025-01-16_02-18-06/PPO_2025-01-16_02-18-06/driver_artifacts
Number of trials: 1/1 (1 PENDING)


== Status ==
Current time: 2025-01-16 02:18:22 (running for 00:00:15.62)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/10 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:L4)
Result logdir: /tmp/ray/session_2025-01-16_02-18-04_666737_42841/artifacts/2025-01-16_02-18-06/PPO_2025-01-16_02-18-06/driver_artifacts
Number of trials: 1/1 (1 PENDING)




...

(PPO(env=open_spiel_env; env-runners=2; learners=1; multi-agent=True) pid=43809) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/PPO_2025-01-16_02-18-06/PPO_open_spiel_env_1f9c6_00000_0_2025-01-16_02-18-07/checkpoint_000041)
(MultiAgentEnvRunner pid=43939) OpenSpiel exception: /project/open_spiel/games/connect_four/connect_four.cc:96 CellAt(kRows - 1, move) == CellState::kEmpty [repeated 27x across cluster]
(MultiAgentEnvRunner pid=43939) CellAt(kRows - 1, move) = X, CellState::kEmpty = Empty [repeated 12x across cluster]
(MultiAgentEnvRunner pid=43938) CellAt(kRows - 1, move) = O, CellState::kEmpty = Empty [repeated 15x across cluster]


== Status ==
Current time: 2025-01-16 02:22:55 (running for 00:04:48.23)
Using FIFO scheduling algorithm.
Logical resource usage: 3.0/10 CPUs, 1.0/1 GPUs (0.0/1.0 accelerator_type:L4)
Result logdir: /tmp/ray/session_2025-01-16_02-18-04_666737_42841/artifacts/2025-01-16_02-18-06/PPO_2025-01-16_02-18-06/driver_artifacts
Number of trials: 1/1 (1 RUNNING)




2025-01-16 02:22:56,996	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/PPO_2025-01-16_02-18-06' in 0.0354s.


(PPO(env=open_spiel_env; env-runners=2; learners=1; multi-agent=True) pid=43809) Iter=43 win-rate=0.97 -> adding new opponent to the mix (main_v1).
Trial PPO_open_spiel_env_1f9c6_00000 reported env_runners/episode_len_mean=9.88,num_env_steps_sampled_lifetime=172000,env_runners/episode_return_mean=0.0 with parameters={'exploration_config': {}, 'extra_python_environs_for_driver': {}, 'extra_python_environs_for_worker': {}, 'placement_strategy': 'PACK', 'num_gpus': 0, '_fake_gpus': False, 'num_cpus_for_main_process': 1, 'eager_tracing': True, 'eager_max_retraces': 20, 'tf_session_args': {'intra_op_parallelism_threads': 2, 'inter_op_parallelism_threads': 2, 'gpu_options': {'allow_growth': True}, 'log_device_placement': False, 'device_count': {'CPU': 1}, 'allow_soft_placement': True}, 'local_tf_session_args': {'intra_op_parallelism_threads': 8, 'inter_op_parallelism_threads': 8}, 'torch_compile_learner': False, 'torch_compile_learner_what_to_compile': <TorchCompileWhatToCompile.FORWARD_TRAI

(PPO(env=open_spiel_env; env-runners=2; learners=1; multi-agent=True) pid=43809) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/root/ray_results/PPO_2025-01-16_02-18-06/PPO_open_spiel_env_1f9c6_00000_0_2025-01-16_02-18-07/checkpoint_000042)
(MultiAgentEnvRunner pid=43938) OpenSpiel exception: /project/open_spiel/games/connect_four/connect_four.cc:96 CellAt(kRows - 1, move) == CellState::kEmpty [repeated 14x across cluster]
(MultiAgentEnvRunner pid=43939) CellAt(kRows - 1, move) = X, CellState::kEmpty = Empty [repeated 9x across cluster]
(MultiAgentEnvRunner pid=43938) CellAt(kRows - 1, move) = O, CellState::kEmpty = Empty [repeated 5x across cluster]
2025-01-16 02:22:57,366	INFO tune.py:1041 -- Total run time: 290.39 seconds (289.97 seconds for the tuning loop).


In [9]:
# Restore trained Algorithm (set to non-explore behavior) and play against
# human on command line.
if args.num_episodes_human_play > 0:
    register_env("open_spiel_env",
                 lambda _: OpenSpielEnv(pyspiel.load_game(args.env)))
    num_episodes = 0
    config.explore = False
    algo = config.build()
    if args.from_checkpoint:
        algo.restore(args.from_checkpoint)
    else:
        checkpoint = results.get_best_result().checkpoint
        if not checkpoint:
            raise ValueError("No last checkpoint found in results!")
        algo.restore(checkpoint)

    if args.enable_new_api_stack:
        rl_module = algo.get_module("main")

    # Play from the command line against the trained agent
    # in an actual (non-RLlib-wrapped) open-spiel env.
    human_player = 1
    env = Environment(args.env)

    while num_episodes < args.num_episodes_human_play:
        print("You play as {}".format("o" if human_player else "x"))
        time_step = env.reset()
        while not time_step.last():
            player_id = time_step.observations["current_player"]
            if player_id == human_player:
                action = ask_user_for_action(time_step)
            else:
                obs = np.array(time_step.observations["info_state"][player_id])
                if args.enable_new_api_stack:
                    action = np.argmax(
                        rl_module.forward_inference(
                            {"obs": torch.from_numpy(obs).unsqueeze(0).float()}
                        )["action_dist_inputs"][0].numpy()
                    )
                else:
                    action = algo.compute_single_action(obs, policy_id="main")
                # In case computer chooses an invalid action, pick a
                # random one.
                legal = time_step.observations["legal_actions"][player_id]
                if action not in legal:
                    action = np.random.choice(legal)
            time_step = env.step([action])
            print(f"\n{env.get_state}")

        print(f"\n{env.get_state}")

        print("End of game!")
        if time_step.rewards[human_player] > 0:
            print("You win")
        elif time_step.rewards[human_player] < 0:
            print("You lose")
        else:
            print("Draw")
        # Switch order of players.
        human_player = 1 - human_player

        num_episodes += 1

    algo.stop()

2025-01-16 02:22:58,764	WARNING deprecation.py:50 -- DeprecationWarning: `build` has been deprecated. Use `AlgorithmConfig.build_algo` instead. This will raise an error in the future!
2025-01-16 02:22:58,766	WARNING algorithm_config.py:4702 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
2025-01-16 02:22:58,770	WARNING unified.py:56 -- Could not instantiate TBXLogger: No module named 'tensorboardX'.


Usage stats collection is enabled by default for nightly wheels. To disable this, run the following command: `ray disable-usage-stats` before starting Ray. See https://docs.ray.io/en/master/cluster/usage-stats.html for more details.


2025-01-16 02:23:00,033	INFO worker.py:1841 -- Started a local Ray instance.
(pid=46255) lz4 not available, disabling sample compression. This will significantly impact RLlib performance. To install lz4, run `pip install lz4`.
(MultiAgentEnvRunner pid=46259) 2025-01-16 02:23:09,073	WARNING rl_module.py:419 -- Could not create a Catalog object for your RLModule! If you are not using the new API stack yet, make sure to switch it off in your config: `config.api_stack(enable_rl_module_and_learner=False, enable_env_runner_and_connector_v2=False)`. All algos use the new stack by default. Ignore this message, if your RLModule does not use a Catalog to build its sub-components.
(MultiAgentEnvRunner pid=46259) 2025-01-16 02:23:09,073	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the fut

You play as o

.......
.......
.......
.......
.......
..x....

Choose an action from [0, 1, 2, 3, 4, 5, 6]:
0

.......
.......
.......
.......
.......
o.x....


.......
.......
.......
.......
..x....
o.x....

Choose an action from [0, 1, 2, 3, 4, 5, 6]:
0

.......
.......
.......
.......
o.x....
o.x....


.......
.......
.......
..x....
o.x....
o.x....

Choose an action from [0, 1, 2, 3, 4, 5, 6]:
2

.......
.......
..o....
..x....
o.x....
o.x....


.......
.......
..o....
..x....
o.x....
o.xx...

Choose an action from [0, 1, 2, 3, 4, 5, 6]:
3

.......
.......
..o....
..x....
o.xo...
o.xx...


.......
.......
..o....
..x....
o.xo...
o.xxx..

Choose an action from [0, 1, 2, 3, 4, 5, 6]:
4

.......
.......
..o....
..x....
o.xoo..
o.xxx..


.......
.......
..o....
..x.x..
o.xoo..
o.xxx..

Choose an action from [0, 1, 2, 3, 4, 5, 6]:
3

.......
.......
..o....
..xox..
o.xoo..
o.xxx..


.......
.......
..o.x..
..xox..
o.xoo..
o.xxx..

Choose an action from [0, 1, 2, 3, 4, 5, 6]:
3

......